# **04** Modeling / Feature Selection

In [1533]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split

import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.vector_ar.vecm import VAR
# from statsmodels.tsa.vector_ar.vecm import coint_johansen
# from statsmodels.tsa.stattools import adfuller
# from statsmodels.tsa.seasonal import seasonal_decompose

# from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [1534]:
# This is a list of all of our routes
routes = pd.read_csv('./data/model/rtm_train_updated.csv')['route'].unique()
print(f'Total Unique Routes: {len(routes)}')
# routes

Total Unique Routes: 404


In [1535]:
# This is a placeholder for the routes -- we plan to make 375 different models
itinerary = 'Albany, NY - Chicago, IL'

In [1536]:
# Read In Data
def df_read(df_file_path, route):
    
    # Import Dataframe 
    df = pd.read_csv(df_file_path)[pd.read_csv(df_file_path)['route'] == route].dropna(axis=1)
    df.index = pd.to_datetime(df['year-month'])
    df = df.drop(columns=['year-month'])
#     print(df.shape)
    
    # Extract Potential Features
    features = sorted(df.drop(columns=['airfare', 'route', 'pop_origin', 'pop_dest', 'origin_lat', 'origin_long', 'dest_lat', 'dest_long']).columns)
    
    remove_options = [i.replace('sf_', "").replace('_diff_1', "").replace('_diff_2', "") for i in features if "sf_" in i]
    
    features = sorted(set(features) - set(remove_options))
    
    return df, features

## Create a dictionary to store train & unseen dataframes as well a their potential features
- **THIS WILL TAKE 20 MINUTES TO RUN**

In [1537]:
# 20 MIN TO RUN!!!
# Create a dictionary storing all of our route train data, route unseen data, and potential features

all_route_dict = {}
for rte in routes: # only looking at 10 routes right now
    route_dict = {}
    route_dict['route'] = rte
    route_dict['train_dataframe'] = df_read('./data/model/rtm_train_updated.csv', route=rte)[0]
    route_dict['test_dataframe'] = df_read('./data/model/rtm_test_updated.csv', route=rte)[0]
    route_dict['potential_features'] = df_read('./data/model/rtm_train_updated.csv', route=rte)[1]
    all_route_dict[rte] = route_dict

# Taking a look at the dictionary -- looking at the keys of a specific route here
print(itinerary)
print()
all_route_dict[itinerary].keys()

Albany, NY - Chicago, IL



dict_keys(['route', 'train_dataframe', 'test_dataframe', 'potential_features'])

### Modeling / Visualization / Analysis Function
- The function below takes in the training dataset & the unseen dataset as well as routes
- It will visualize the results of our model for the test dataset as well as predictions on unseen data
- Additionally it will show us R2, RMSE, and MSE data to compare model/results

In [1538]:
def model_optimize_function(itinerary_route='Tampa, FL - Washington, DC', target_variable='airfare', pv_thresh=0.05):
    
    df = all_route_dict[itinerary_route]['train_dataframe']
    unseen_df = all_route_dict[itinerary_route]['test_dataframe']
    features_list = all_route_dict[itinerary_route]['potential_features']
    
    X = df[features_list]
    y = df[target_variable]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, shuffle=False)
    X_train = sm.add_constant(X_train, has_constant='add')          
    X_test = sm.add_constant(X_test, has_constant='add')        
    X_train.dropna(inplace=True)                                     # dropping NA values to ensure model computes
    y_train = y_train[X_train.index]                                 # Subset y to match the index from X
    linear_model = sm.OLS(y_train, X_train).fit()
    
    X_unseen = unseen_df[features_list]                          # Create X variable with matching features from Traing Data
    X_unseen = sm.add_constant(X_unseen, has_constant='add')         # Add X Constant
    unseen_pred = linear_model.predict(X_unseen)                     # Make Predictions -- for 'airfare'
    unseen_df['airfare_predictions'] = unseen_pred   # save predictions to unseen route df
    unseen_df = unseen_df.dropna() 
    
    count = 1
    # Plot Train/Test & Model Predictions on Training Dataset
    plt.figure(figsize=(12, 4))                                                                # Set figure size
    plt.plot(y_train.index, y_train.values, color = 'blue', label='Training Values')           # Plot training data
    plt.plot(y_test.index, y_test.values, color = 'orange', label='Test Values', linewidth=10) # Plot testing data
    plt.plot(linear_model.predict(X_test), color = 'green', label='Predicted Test Values')                     # Plot predicted test values
    plt.title(label = 'Train / Test + Predictions - Model_' + str(count) + ': ' + str(itinerary_route), fontsize=20)            # Plot Title
    plt.ylabel('Airfare Ticket Price ($ USD)', size=12)                                        # Y Label
    plt.xlabel('Year-Month', size=12)                                                          # X label
    plt.grid()                                                                                 # add grid
    plt.xticks(fontsize=12)                                                                    # Resize X-Ticks
    plt.yticks(fontsize=12)                                                                    # Resize Y-Ticks
    plt.legend()                                                                               # Legend
    plt.savefig('./visualizations/' + itinerary_route + '_train_' + str(count) + '.png', transparent=True, bbox_inches = "tight")
    plt.close();    
    
    # Plot Predictions on our unseen dataframe
    plt.figure(figsize=(12, 4)) # Set figure size
    plt.plot(unseen_df[target_variable], color = 'orange', label='Actual Airfare Price', linewidth=10)    # Plot actual data.
    plt.plot(unseen_df['airfare_predictions'], color = 'green', label='Predicted Airfare Price')    # Plot predicted unseen values
    plt.title(label = 'Predictions (unseen data) - Model_' + str(count) + ': ' + str(itinerary_route), fontsize=20) # add title
    plt.ylabel('Airfare Ticket Price ($ USD)', size=12)
    plt.xlabel('Year-Month', size=12)
    plt.grid()
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.legend()
    plt.savefig('./visualizations/' + itinerary_route + '_unseen_' + str(count) + '.png', transparent=True, bbox_inches = "tight")
    plt.close();    
    
    
    ###################
    model_dict = {}
    scoring_dict = {}
    score_dict = {}
    model_dict['lin_model_' + str(count)] = linear_model
    
    # ANALYTICS
#     score_dict['y_train_pred'] = linear_model.predict(X_train) # Generate Predictions
#     score_dict['y_test_pred'] = linear_model.predict(X_test) # Generate Predictions
#     score_dict['y_test_baseline'] = [y_test.mean()] * len(y_test)  # Baseline
    score_dict['train_r2'] = r2_score(y_train, linear_model.predict(X_train))  # Scoring
    score_dict['train_rmse'] = mean_squared_error(y_train, linear_model.predict(X_train), squared=False) # Scoring
    score_dict['test_r2'] = r2_score(y_test, linear_model.predict(X_test)) # Scoring
    score_dict['test_rmse'] = mean_squared_error(y_test, linear_model.predict(X_test), squared=False) # Scoring
    score_dict['base_r2'] = r2_score(y_test, [y_test.mean()] * len(y_test)) # Scoring
    score_dict['base_rmse'] = mean_squared_error(y_test, [y_test.mean()] * len(y_test), squared=False) # Scoring
    score_dict['unseen_r2'] = r2_score(unseen_df[target_variable], unseen_df.airfare_predictions) # Scoring
    score_dict['unseen_rmse'] = mean_squared_error(unseen_df[target_variable], unseen_df.airfare_predictions, squared=False) # Scoring
    
    scoring_dict['analytics_' + str(count)] = score_dict
#     score_dict['analytics_' + str(count)] = score_dict
    ###################
    
    model_dict = {}
    scoring_dict = {}
    score_dict = {}

    removed_features_list = []
    condition = linear_model.pvalues[linear_model.pvalues.index != 'const'].sort_values(ascending=False).max()
    
    while condition > pv_thresh:
        count +=1
        removed_features_list.append(linear_model.pvalues[linear_model.pvalues.index != 'const'].sort_values(ascending=False).index[0])
#         print(f'Removed: {linear_model.pvalues.sort_values(ascending=False).index[0]}')
        features_list = list(set(features_list) - set(removed_features_list))
        
        X = df[features_list]
        y = df['airfare']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, shuffle=False)
        X_train = sm.add_constant(X_train, has_constant='add')          
        X_test = sm.add_constant(X_test, has_constant='add')        
        X_train.dropna(inplace=True)                                     # dropping NA values to ensure model computes
        y_train = y_train[X_train.index]                                 # Subset y to match the index from X
        linear_model = sm.OLS(y_train, X_train).fit()
        
        condition = linear_model.pvalues.sort_values(ascending=False).max()
        
        X_unseen = unseen_df[features_list]                          # Create X variable with matching features from Traing Data
        X_unseen = sm.add_constant(X_unseen, has_constant='add')         # Add X Constant
        unseen_pred = linear_model.predict(X_unseen)                     # Make Predictions -- for 'airfare'
        unseen_df['airfare_predictions'] = unseen_pred   # save predictions to unseen route df
        unseen_df = unseen_df.dropna() 
        
        score_dict = {}
        # ANALYTICS
#         score_dict['y_train_pred'] = linear_model.predict(X_train) # Generate Predictions
#         score_dict['y_test_pred'] = linear_model.predict(X_test) # Generate Predictions
#         score_dict['y_test_baseline'] = [y_test.mean()] * len(y_test)  # Baseline
        score_dict['train_r2'] = r2_score(y_train, linear_model.predict(X_train))  # Scoring
        score_dict['train_rmse'] = mean_squared_error(y_train, linear_model.predict(X_train), squared=False) # Scoring
        score_dict['test_r2'] = r2_score(y_test, linear_model.predict(X_test)) # Scoring
        score_dict['test_rmse'] = mean_squared_error(y_test, linear_model.predict(X_test), squared=False) # Scoring
        score_dict['base_r2'] = r2_score(y_test, [y_test.mean()] * len(y_test)) # Scoring
        score_dict['base_rmse'] = mean_squared_error(y_test, [y_test.mean()] * len(y_test), squared=False) # Scoring
        score_dict['unseen_r2'] = r2_score(unseen_df[target_variable], unseen_df.airfare_predictions) # Scoring
        score_dict['unseen_rmse'] = mean_squared_error(unseen_df[target_variable], unseen_df.airfare_predictions, squared=False) # Scoring
        scoring_dict['analytics_' + str(count)] = score_dict
        ###################
 
        model_dict['lin_model_' + str(count)] = linear_model
    
    all_route_dict[itinerary_route]['models'] = model_dict
    all_route_dict[itinerary_route]['analytics'] = scoring_dict
    all_route_dict[itinerary_route]['final_features'] = features_list
    all_route_dict[itinerary_route]['final_model'] = linear_model
    
    # Plot Train/Test & Model Predictions on Training Dataset
    plt.figure(figsize=(12, 4))                                                                # Set figure size
    plt.plot(y_train.index, y_train.values, color = 'blue', label='Training Values')           # Plot training data
    plt.plot(y_test.index, y_test.values, color = 'orange', label='Test Values', linewidth=10) # Plot testing data
    plt.plot(linear_model.predict(X_test), color = 'green', label='Predicted Test Values')                     # Plot predicted test values
    plt.title(label = 'Train / Test + Predictions - Model_' + str(count) + ': ' + str(itinerary_route), fontsize=20)            # Plot Title
    plt.ylabel('Airfare Ticket Price ($ USD)', size=12)                                        # Y Label
    plt.xlabel('Year-Month', size=12)                                                          # X label
    plt.grid()                                                                                 # add grid
    plt.xticks(fontsize=12)                                                                    # Resize X-Ticks
    plt.yticks(fontsize=12)                                                                    # Resize Y-Ticks
    plt.legend()                                                                               # Legend
    plt.savefig('./visualizations/' + itinerary_route + '_train_' + str(count) + '.png', transparent=True, bbox_inches = "tight")
    plt.close();    
    
    # Plot Predictions on our unseen dataframe
    plt.figure(figsize=(12, 4)) # Set figure size
    plt.plot(unseen_df[target_variable], color = 'orange', label='Actual Airfare Price', linewidth=10)    # Plot actual data.
    plt.plot(unseen_df['airfare_predictions'], color = 'red', label='Predicted Airfare Price')    # Plot predicted unseen values
    plt.title(label = 'Predictions (unseen data) - Model_' + str(count) + ': ' + str(itinerary_route), fontsize=20) # add title
    plt.ylabel('Airfare Ticket Price ($ USD)', size=12)
    plt.xlabel('Year-Month', size=12)
    plt.grid()
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.legend()
    plt.savefig('./visualizations/' + itinerary_route + '_unseen_final_' + str(count) + '.png', transparent=True, bbox_inches = "tight")
    plt.close();                  

# Let's apply our function to all of our data
- Let's leverage our functions and run them through a loop of all our our routes
- Each route will have its own custom fit model to make predictions
- Output will be visualizations and scores
- Next Steps:
    - Analyze stats and tune function to output best models

In [1539]:
for rte in list(all_route_dict.keys()):
    model_optimize_function(itinerary_route=rte)
    plt.rcParams.update({'figure.max_open_warning': 0})

## Now Create a Dataframe with the results

In [1540]:
# Now let's create a dataframe with all of our results

answers_df = pd.DataFrame()
for i in list(all_route_dict.keys()):
    rte_df = pd.DataFrame()
    for j in list(all_route_dict[i]['analytics'].keys()):
        new_dataframe = pd.DataFrame(all_route_dict[i]['analytics'][j], index=[0])
        new_dataframe['analytics_count'] = j
        new_dataframe['route'] = i
        rte_df = pd.concat([rte_df, new_dataframe])
    answers_df = pd.concat([answers_df, rte_df])
   
other_df = pd.DataFrame()
for i in list(all_route_dict.keys()):
    newly_df = pd.DataFrame.from_dict(all_route_dict[i]['models'], orient='index')
    other_df = pd.concat([other_df, newly_df])

print(answers_df.shape)
print(other_df.shape)

answers_df.reset_index(inplace=True)
other_df.reset_index(inplace=True)

# Merge the two DF's above
scoring_df = pd.merge(other_df, answers_df, left_index=True, right_index=True)
scoring_df = scoring_df[['index_x', 0, 'route', 'base_r2', 'train_r2', 'test_r2', 'unseen_r2', 'base_rmse', 'train_rmse', 'test_rmse','unseen_rmse']].rename(columns={0 : 'linear_model', 'index_x' : 'model_order'})

# Scoring Analytics --------------------------

#Change Model Order column to integer
scoring_df['model_order'] = scoring_df['model_order'].apply(lambda x: int(x.replace('lin_model_', "")))

# Train VS Test
scoring_df['train_r2_overfit'] = scoring_df['train_r2'] > scoring_df['test_r2']
scoring_df['train_rmse_overfit'] = scoring_df['train_rmse'] < scoring_df['test_rmse']

# Unseen VS Model Predictions
scoring_df['unseen_r2_overfit'] = scoring_df['unseen_r2'] > scoring_df['test_r2']
scoring_df['unseen_rmse_overfit'] = scoring_df['unseen_rmse'] < scoring_df['test_rmse']


print(scoring_df.shape)
scoring_df.head()

(8474, 10)
(8474, 1)
(8474, 15)


,model_order,linear_model,route,base_r2,train_r2,test_r2,unseen_r2,base_rmse,train_rmse,test_rmse,unseen_rmse,train_r2_overfit,train_rmse_overfit,unseen_r2_overfit,unseen_rmse_overfit
0,2,<statsmodels.regression.linear_model.Regressio...,"Albany, NY - Chicago, IL",-2.220446e-16,0.765730,-11.211441,-97.827997,14.673704,21.095879,51.277072,99.984673,True,True,False,False
1,3,<statsmodels.regression.linear_model.Regressio...,"Albany, NY - Chicago, IL",-2.220446e-16,0.765729,-11.236172,-98.036607,14.673704,21.095937,51.328969,100.090144,True,True,False,False
2,4,<statsmodels.regression.linear_model.Regressio...,"Albany, NY - Chicago, IL",-2.220446e-16,0.765719,-11.321700,-98.923642,14.673704,21.096368,51.508045,100.537380,True,True,False,False
3,5,<statsmodels.regression.linear_model.Regressio...,"Albany, NY - Chicago, IL",-2.220446e-16,0.765704,-11.312178,-96.060562,14.673704,21.097065,51.488139,99.086580,True,True,False,False
4,6,<statsmodels.regression.linear_model.Regressio...,"Albany, NY - Chicago, IL",-2.220446e-16,0.765672,-11.532504,-97.750906,14.673704,21.098491,51.946786,99.945669,True,True,False,False


In [1541]:
# 265 out of 375 testing scores outperformed
realquick = scoring_df.groupby('route')[['base_r2', 'train_r2', 'test_r2']].max()
realquick['greater'] = realquick['test_r2'] > realquick['base_r2']
realquick['greater'].value_counts()

False    283
True     121
Name: greater, dtype: int64

## LET'S TAKE A LOOK AT OUR MOST POPULAR ROUTE!

### ROUTE: Dallas, TX - Houston, TX

In [1544]:
all_route_dict['Dallas, TX - Houston, TX']['analytics'].keys()
all_route_dict['Dallas, TX - Houston, TX']['analytics']['analytics_2']
all_route_dict['Dallas, TX - Houston, TX']['analytics']['analytics_21']

# final analytics

{'train_r2': 0.3356061729191988,
 'train_rmse': 4.3807916320680835,
 'test_r2': -13.134128246142426,
 'test_rmse': 15.899739689129524,
 'base_r2': 2.220446049250313e-16,
 'base_rmse': 4.229173481111092,
 'unseen_r2': -198.2221467339791,
 'unseen_rmse': 36.39835549646081}

In [1545]:
# final features
all_route_dict['Dallas, TX - Houston, TX']['final_features']

['sf_cost_per_mile_diff_1',
 'sf_flight_revenue_diff_2',
 'sf_airfare_diff_1',
 'sf_time_diff_1',
 'sf_passengers_diff_2',
 'sf_total_flight_cost_diff_2',
 'dist_miles']

In [1546]:
# final model
all_route_dict['Dallas, TX - Houston, TX']['final_model'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                airfare   R-squared:                       0.336
Model:                            OLS   Adj. R-squared:                  0.282
Method:                 Least Squares   F-statistic:                     6.314
Date:                Mon, 12 Oct 2020   Prob (F-statistic):           2.08e-05
Time:                        20:30:53   Log-Likelihood:                -237.49
No. Observations:                  82   AIC:                             489.0
Df Residuals:                      75   BIC:                             505.8
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                          -0.0030      0.001     -4.711      0.000      -0.004      -0.002
sf_cost_per_mile_diff_1       911.6358    347.708      2.622      0.011     218.966    1604.306
sf_flight_revenue_diff_2     4.358e-06   2.08e-06      2.099      0.039    2.22e-07    8.49e-06
sf_airfare_diff_1              -4.0965      1.787     -2.293      0.025      -7.656      -0.537
sf_time_diff_1                 -1.2160      0.258     -4.711      0.000      -1.730      -0.702
sf_passengers_diff_2           -0.0004      0.000     -2.159      0.034      -0.001   -2.92e-05
sf_total_flight_cost_diff_2  2.335e-05   1.04e-05      2.255      0.027    2.73e-06     4.4e-05
dist_miles                      2.5084      0.454      5.523      0.000       1.604       3.413
==============================================================================
Omnibus:                        2.809   Durbin-Watson:                   0.181
Prob(Omnibus):                  0.246   Jarque-Bera (JB):                2.408
Skew:                           0.419   Prob(JB):                        0.300
Kurtosis:                       3.050   Cond. No.                     9.55e+21
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.9e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

- Immediately we are outperfroming our Baseline Scores, but not by much given it's a low cost ticket
- Looking at how poorly this performs against unseen data we could not move forward with this route specific model.

<img src='./Keeping_Visualizations/DallasTXHoustonTX_train_1.png'>

<img src='.//Keeping_Visualizations/Dallas, TX - Houston, TX_unseen_1.png'>

<img src='./Keeping_Visualizations/Dallas, TX - Houston, TX_train_22.png'>

<img src='./Keeping_Visualizations/Dallas, TX - Houston, TX_unseen_final_22.png'>

## NOW LET'S TAKE A LOOK AT ANOTHER (MORE SUCCESSFUL) ROUTE

### ROUTE: Atlanta, GA - Austin, TX

In [1547]:
all_route_dict['Atlanta, GA - Austin, TX']['analytics'].keys()
all_route_dict['Atlanta, GA - Austin, TX']['analytics']['analytics_2']
all_route_dict['Atlanta, GA - Austin, TX']['analytics']['analytics_21']

# final analytics

{'train_r2': 0.9821004334857656,
 'train_rmse': 2.5799529950916957,
 'test_r2': 0.9727441351175675,
 'test_rmse': 4.951731579481935,
 'base_r2': 0.0,
 'base_rmse': 29.99349747815719,
 'unseen_r2': 0.9454592031032688,
 'unseen_rmse': 6.640727221172836}

In [1548]:
# final features
all_route_dict['Atlanta, GA - Austin, TX']['final_features']

['sf_total_flight_miles_diff_1',
 'sf_num_of_flights_lag_12_diff_1',
 'sf_time_diff_1',
 'seat_capacity_lag_2',
 'passengers',
 'dist_miles',
 'flight_revenue']

In [1549]:
# final model
all_route_dict['Atlanta, GA - Austin, TX']['final_model'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                airfare   R-squared:                       0.982
Model:                            OLS   Adj. R-squared:                  0.981
Method:                 Least Squares   F-statistic:                     685.8
Date:                Mon, 12 Oct 2020   Prob (F-statistic):           2.23e-63
Time:                        20:30:53   Log-Likelihood:                -194.07
No. Observations:                  82   AIC:                             402.1
Df Residuals:                      75   BIC:                             419.0
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const                              -0.0122      0.005     -2.653      0.010      -0.021      -0.003
sf_total_flight_miles_diff_1        0.0001   3.66e-05      3.655      0.000    6.08e-05       0.000
sf_num_of_flights_lag_12_diff_1     0.1031      0.025      4.054      0.000       0.052       0.154
sf_time_diff_1                     -4.9393      1.862     -2.653      0.010      -8.648      -1.231
seat_capacity_lag_2                 0.0005      0.000      4.309      0.000       0.000       0.001
passengers                         -0.0151      0.000    -43.684      0.000      -0.016      -0.014
dist_miles                          2.7425      0.927      2.959      0.004       0.896       4.589
flight_revenue                   6.142e-05   1.14e-06     54.072      0.000    5.92e-05    6.37e-05
==============================================================================
Omnibus:                        9.540   Durbin-Watson:                   0.997
Prob(Omnibus):                  0.008   Jarque-Bera (JB):                9.485
Skew:                          -0.810   Prob(JB):                      0.00872
Kurtosis:                       3.390   Cond. No.                     3.99e+21
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 7.02e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

- Immediately we are outperfroming our Baseline Scores, however tuning this model leads us to see no improvement.
- But do you really need to see improvement when you are scoring 98%.  A good time series model has minimal features they say!
- Given the price ranges roughly from 90-105 USD, this model is deemed a success performing well on unseen data!.
- Would highly encourage an individual to book travel that was listed below this as you would be finding a bargain!

**HOWEVER! Note the model performs poorly on unseen data!**

<img src='./Keeping_Visualizations/Atlanta, GA - Austin, TX_train_1.png'>

<img src='./Keeping_Visualizations/Atlanta, GA - Austin, TX_unseen_1.png'>

<img src='.//Keeping_Visualizations/Atlanta, GA - Austin, TX_train_21.png'>

<img src='./Keeping_Visualizations/Atlanta, GA - Austin, TX_unseen_final_21.png'>

## - Able to reduce average RMSE by 21.85% -- Improved Score on 299 out of 375 routes
## - Success with 89 out of 375 routes in predicting airfare prices near perfect r2


In [1550]:
join_score = scoring_df.groupby('route')[['model_order']].max().reset_index()
join_score.head()

,route,model_order
0,"Albany, NY - Chicago, IL",25
1,"Albany, NY - Orlando, FL",18
2,"Albuquerque, NM - Chicago, IL",22
3,"Albuquerque, NM - Dallas, TX",26
4,"Albuquerque, NM - Houston, TX",26


In [1551]:
# make dataframe of initial scores
initial_score_df = scoring_df[scoring_df['model_order'] == 2]
initial_score_df.head(2)

,model_order,linear_model,route,base_r2,train_r2,test_r2,unseen_r2,base_rmse,train_rmse,test_rmse,unseen_rmse,train_r2_overfit,train_rmse_overfit,unseen_r2_overfit,unseen_rmse_overfit
0,2,<statsmodels.regression.linear_model.Regressio...,"Albany, NY - Chicago, IL",-2.220446e-16,0.765730,-11.211441,-97.827997,14.673704,21.095879,51.277072,99.984673,True,True,False,False
24,2,<statsmodels.regression.linear_model.Regressio...,"Albany, NY - Orlando, FL",0.000000e+00,0.531803,-1.399643,-34.334557,10.436395,9.252736,16.166790,47.417078,True,True,False,False


In [1552]:
# make dataframe of final scores
# Merge join score dataframe to showcase the final statistics for each of the final models per route
final_score_df = join_score.merge(scoring_df, left_on=['model_order', 'route'], right_on=['model_order', 'route'], how='left')
final_score_df.head(2)

,route,model_order,linear_model,base_r2,train_r2,test_r2,unseen_r2,base_rmse,train_rmse,test_rmse,unseen_rmse,train_r2_overfit,train_rmse_overfit,unseen_r2_overfit,unseen_rmse_overfit
0,"Albany, NY - Chicago, IL",25,<statsmodels.regression.linear_model.Regressio...,-2.220446e-16,0.732433,-5.166856,-45.947541,14.673704,22.545295,36.439436,68.912794,True,True,False,False
1,"Albany, NY - Orlando, FL",18,<statsmodels.regression.linear_model.Regressio...,0.000000e+00,0.416107,-0.692976,-26.473834,10.436395,10.332903,13.579255,41.811389,True,True,False,False


## Now let's look to see how we improved!

In [1553]:
# How many models outperformed the baseline models initially?
len(initial_score_df[initial_score_df['base_r2'] < initial_score_df['test_r2']])

101

In [1554]:
# How many model predictions outperformed the baseline model?
len(initial_score_df[(initial_score_df['base_r2'] < initial_score_df['test_r2']) & (initial_score_df['base_r2'] < initial_score_df['unseen_r2'])])

81

- Looking at the above 93 of our models outperformed the baseline model
- Of those 93 models, 75 of the predictions made on unseen data outperforms the baseline models!

In [1555]:
initial_score_df.columns

Index(['model_order', 'linear_model', 'route', 'base_r2', 'train_r2',
       'test_r2', 'unseen_r2', 'base_rmse', 'train_rmse', 'test_rmse',
       'unseen_rmse', 'train_r2_overfit', 'train_rmse_overfit',
       'unseen_r2_overfit', 'unseen_rmse_overfit'],
      dtype='object')

In [1556]:
# Let's look at the before & after data
initial = initial_score_df.agg(['mean']).T #2nd row
final = final_score_df.agg(['mean']).T #ast row
compare = pd.concat([initial, final], axis=1)
compare.columns = ['initial', 'final']
compare

,initial,final
model_order,2.000000e+00,2.197525e+01
base_r2,5.496154e-19,5.496154e-19
train_r2,6.404712e-01,5.151762e-01
test_r2,-1.132155e+01,-5.721649e+00
unseen_r2,-4.734430e+01,-2.555999e+01
base_rmse,1.730232e+01,1.730232e+01
train_rmse,1.094662e+01,1.282957e+01
test_rmse,3.467619e+01,2.906967e+01
unseen_rmse,5.145863e+01,4.105996e+01
train_r2_overfit,9.158416e-01,9.232673e-01


- through feature selection optimizations...
    - R2
        - train R2 from 0.64 -->> 0.53
        - test R2 from -7.54 -->> -4.70
        - unseen R2 from -48.0 -->> -25.2
    - RMSE
        - train RMSE from 11.30 -->> 12.97
        - test RMSE from 34.26 -->> 28.96
        - unseen RMSE from 51.76 -->> 40.18

### Let's look at the routes closer and check out how each scored

##### Train (from train/test)

In [1557]:
# Let's look at the routes and their scoring!
count = 0
for i in range(0, 10):
    length = len(final_score_df[(final_score_df['train_r2'] > (i/10)) & (final_score_df['train_r2'] <= ((i+1)/10))]['route'].unique())
    print(f'R2 Score {int((i/10)*100)}-{int((i+1)/10*100)}%: {length} routes')
    count += length
    print()
    print(final_score_df[(final_score_df['train_r2'] > (i/10)) & (final_score_df['train_r2'] <= ((i+1)/10))]['route'].unique())
    print()
    i_count += i

train_above_1 = len(final_score_df[final_score_df['train_r2'] > 1])
train_below_0 = len(final_score_df[final_score_df['train_r2'] <= 0])
print(f'{train_above_1} routes above 100%')
print(f'{train_below_0} routes below 0%')
print(f'{count} total routes scoring between 0-100%')

R2 Score 0-10%: 20 routes

['Atlanta, GA - Boston, MA' 'Boston, MA - New York, NY'
 'Chicago, IL - Nashville, TN' 'Chicago, IL - Omaha, NE'
 'Chicago, IL - St. Louis, MO' 'Cincinnati, OH - Philadelphia, PA'
 'Cleveland, OH - Los Angeles, CA' 'Dallas, TX - El Paso, TX'
 'Dallas, TX - Kansas City, MO' 'Detroit, MI - St. Louis, MO'
 'Houston, TX - Oklahoma City, OK' 'Las Vegas, NV - Los Angeles, CA'
 'Las Vegas, NV - Minneapolis, MN' 'Los Angeles, CA - Phoenix, AZ'
 'Los Angeles, CA - Reno, NV' 'Memphis, TN - Orlando, FL'
 'Minneapolis, MN - Orlando, FL' 'Nashville, TN - Tampa, FL'
 'Salt Lake City, UT - St. Louis, MO' 'Seattle, WA - Spokane, WA']

R2 Score 10-20%: 42 routes

['Atlanta, GA - Memphis, TN' 'Boston, MA - Dallas, TX'
 'Charlotte, NC - Dallas, TX' 'Chicago, IL - Las Vegas, NV'
 'Chicago, IL - Minneapolis, MN' 'Chicago, IL - Portland, OR'
 'Chicago, IL - Tucson, AZ' 'Cincinnati, OH - Orlando, FL'
 'Dallas, TX - Jacksonville, FL' 'Dallas, TX - Milwaukee, WI'
 'Dallas, TX - Omaha

##### Test (from train/test)

In [1558]:
# Let's look at the routes and their scoring!
count = 0
for i in range(0, 10):
    length = len(final_score_df[(final_score_df['test_r2'] > (i/10)) & (final_score_df['test_r2'] <= ((i+1)/10))]['route'].unique())
    print(f'R2 Score {int((i/10)*100)}-{int((i+1)/10*100)}%: {length} routes')
    count += length
    print()
    print(final_score_df[(final_score_df['test_r2'] > (i/10)) & (final_score_df['test_r2'] <= ((i+1)/10))]['route'].unique())
    print()
    i_count += i

test_above_1 = len(final_score_df[final_score_df['test_r2'] > 1])
test_below_0 = len(final_score_df[final_score_df['test_r2'] <= 0])
print(f'{test_above_1} routes above 100%')
print(f'{test_below_0} routes below 0%')
print(f'{count} total routes scoring between 0-100%')

R2 Score 0-10%: 4 routes

['Atlanta, GA - Portland, OR' 'Indianapolis, IN - Minneapolis, MN'
 'Las Vegas, NV - Philadelphia, PA' 'Los Angeles, CA - Sacramento, CA']

R2 Score 10-20%: 8 routes

['Atlanta, GA - Washington, DC' 'Columbus, OH - New York, NY'
 'Dallas, TX - Detroit, MI' 'El Paso, TX - Los Angeles, CA'
 'Jacksonville, FL - Philadelphia, PA' 'Miami, FL - New York, NY'
 'Milwaukee, WI - Washington, DC' 'Phoenix, AZ - Reno, NV']

R2 Score 20-30%: 1 routes

['Boston, MA - Pittsburgh, PA']

R2 Score 30-40%: 6 routes

['Atlanta, GA - Orlando, FL' 'Austin, TX - Las Vegas, NV'
 'Austin, TX - Nashville, TN' 'Detroit, MI - Orlando, FL'
 'Los Angeles, CA - Seattle, WA' 'Salt Lake City, UT - Washington, DC']

R2 Score 40-50%: 6 routes

['Charlotte, NC - Minneapolis, MN' 'Chicago, IL - Las Vegas, NV'
 'Detroit, MI - Miami, FL' 'Hartford, CT - Orlando, FL'
 'Minneapolis, MN - Raleigh, NC' 'Sacramento, CA - Seattle, WA']

R2 Score 50-60%: 3 routes

['Atlanta, GA - Chicago, IL' 'Dallas, TX 

##### Unseen Data

In [1559]:
# Let's look at the routes and their scoring!
count = 0
for i in range(0, 10):
    length = len(final_score_df[(final_score_df['unseen_r2'] > (i/10)) & (final_score_df['unseen_r2'] <= ((i+1)/10))]['route'].unique())
    print(f'R2 Score {int((i/10)*100)}-{int((i+1)/10*100)}%: {length} routes')
    count += length
    print()
    print(final_score_df[(final_score_df['unseen_r2'] > (i/10)) & (final_score_df['unseen_r2'] <= ((i+1)/10))]['route'].unique())
    print()
    i_count += i

unseen_above_1 = len(final_score_df[final_score_df['unseen_r2'] > 1])
unseen_below_0 = len(final_score_df[final_score_df['unseen_r2'] <= 0])
print(f'{unseen_above_1} routes above 100%')
print(f'{unseen_below_0} routes below 0%')
print(f'{count} total routes scoring between 0-100%')

R2 Score 0-10%: 6 routes

['Boston, MA - Raleigh, NC' 'Cincinnati, OH - Los Angeles, CA'
 'Cincinnati, OH - Orlando, FL' 'Dallas, TX - Raleigh, NC'
 'Kansas City, MO - New York, NY' 'Memphis, TN - New York, NY']

R2 Score 10-20%: 5 routes

['Chicago, IL - Cincinnati, OH' 'Chicago, IL - Tampa, FL'
 'Indianapolis, IN - Minneapolis, MN' 'Miami, FL - Washington, DC'
 'Orlando, FL - San Francisco, CA']

R2 Score 20-30%: 2 routes

['Indianapolis, IN - New York, NY' 'Minneapolis, MN - Nashville, TN']

R2 Score 30-40%: 1 routes

['Houston, TX - Minneapolis, MN']

R2 Score 40-50%: 1 routes

['Boston, MA - Los Angeles, CA']

R2 Score 50-60%: 1 routes

['Chicago, IL - Raleigh, NC']

R2 Score 60-70%: 1 routes

['Atlanta, GA - Chicago, IL']

R2 Score 70-80%: 0 routes

[]

R2 Score 80-90%: 2 routes

['Los Angeles, CA - Orlando, FL' 'Orlando, FL - Salt Lake City, UT']

R2 Score 90-100%: 62 routes

['Albuquerque, NM - Dallas, TX' 'Albuquerque, NM - Houston, TX'
 'Atlanta, GA - Austin, TX' 'Atlanta, GA

In [1560]:
# Let's look at where we started and where we ended!

initial_train_r2 = []
initial_train_rmse = []
for path in list(all_route_dict.keys()):
    if all_route_dict[path]['analytics'][list(all_route_dict[path]['analytics'].keys())[0]]['train_r2'] > 0:
        initial_train_r2.append(all_route_dict[path]['analytics'][list(all_route_dict[path]['analytics'].keys())[0]]['train_r2'])
        initial_train_rmse.append(all_route_dict[path]['analytics'][list(all_route_dict[path]['analytics'].keys())[0]]['train_rmse'])
    
final_train_r2 = []
final_train_rmse = []
for path in list(all_route_dict.keys()):
    if all_route_dict[path]['analytics'][list(all_route_dict[path]['analytics'].keys())[-1]]['train_r2'] > 0:
        final_train_r2.append(all_route_dict[path]['analytics'][list(all_route_dict[path]['analytics'].keys())[-1]]['train_r2'])
        final_train_rmse.append(all_route_dict[path]['analytics'][list(all_route_dict[path]['analytics'].keys())[-1]]['train_rmse'])
        
initial_test_r2 = []
initial_test_rmse = []
for path in list(all_route_dict.keys()):
    if all_route_dict[path]['analytics'][list(all_route_dict[path]['analytics'].keys())[0]]['test_r2'] > 0:
        initial_test_r2.append(all_route_dict[path]['analytics'][list(all_route_dict[path]['analytics'].keys())[0]]['test_r2'])
        initial_test_rmse.append(all_route_dict[path]['analytics'][list(all_route_dict[path]['analytics'].keys())[0]]['test_rmse'])
    
final_test_r2 = []
final_test_rmse = []
for path in list(all_route_dict.keys()):
    if all_route_dict[path]['analytics'][list(all_route_dict[path]['analytics'].keys())[-1]]['test_r2'] > 0:
        final_test_r2.append(all_route_dict[path]['analytics'][list(all_route_dict[path]['analytics'].keys())[-1]]['test_r2'])
        final_test_rmse.append(all_route_dict[path]['analytics'][list(all_route_dict[path]['analytics'].keys())[-1]]['test_rmse'])
    

- Looking at the changes from where we started to where we went, you'll notice scores did in fact go up, however, as noted earlier we exceeding baseline scores significantly off of the bat.  In the future we should look to find better ways to tune our model in order to reduce overfitting!

In [1561]:
print(f'Overall Initial Train R2 Average: {np.mean(initial_train_r2)}')
print(f'Overall Final Train R2 Average: {np.mean(final_train_r2)}')
print(f'Overall Initial Test R2 Average: {np.mean(initial_test_r2)}')
print(f'Overall Final Test R2 Average: {np.mean(final_test_r2)}')
print()
print(f'Overall Initial Train RMSE Average: {np.mean(initial_train_rmse)}')
print(f'Overall Final Train RMSE Average: {np.mean(final_train_rmse)}')
print(f'Overall Initial Test RMSE Average: {np.mean(initial_test_rmse)}')
print(f'Overall Final Test RMSE Average: {np.mean(final_test_rmse)}')

Overall Initial Train R2 Average: 0.6404712401892289
Overall Final Train R2 Average: 0.5462760462962177
Overall Initial Test R2 Average: 0.8479141175858956
Overall Final Test R2 Average: 0.7757444879470826

Overall Initial Train RMSE Average: 10.9466150305228
Overall Final Train RMSE Average: 12.226174003620205
Overall Initial Test RMSE Average: 3.395936479562116
Overall Final Test RMSE Average: 5.290230018045353


In [1562]:
# Another way to view our scores by route

route_scores_dict = {}
for path in list(all_route_dict.keys()):
    only_dict = {}
    only_dict['route'] = path
    only_dict['initial_train_r2'] = all_route_dict[path]['analytics'][list(all_route_dict[path]['analytics'].keys())[0]]['train_r2']
    only_dict['final_train_r2'] = all_route_dict[path]['analytics'][list(all_route_dict[path]['analytics'].keys())[-1]]['train_r2']
    only_dict['initial_test_r2'] = all_route_dict[path]['analytics'][list(all_route_dict[path]['analytics'].keys())[0]]['test_r2']
    only_dict['final_test_r2'] = all_route_dict[path]['analytics'][list(all_route_dict[path]['analytics'].keys())[-1]]['test_r2']
    only_dict['initial_train_rmse'] = all_route_dict[path]['analytics'][list(all_route_dict[path]['analytics'].keys())[0]]['train_rmse']
    only_dict['final_train_rmse'] = all_route_dict[path]['analytics'][list(all_route_dict[path]['analytics'].keys())[-1]]['train_rmse']
    only_dict['initial_test_rmse'] = all_route_dict[path]['analytics'][list(all_route_dict[path]['analytics'].keys())[0]]['test_rmse']
    only_dict['final_test_rmse'] = all_route_dict[path]['analytics'][list(all_route_dict[path]['analytics'].keys())[-1]]['test_rmse']
    route_scores_dict[path] = only_dict

only_df = pd.DataFrame(route_scores_dict).T.reset_index().drop(columns='index')
only_df

,route,initial_train_r2,final_train_r2,initial_test_r2,final_test_r2,initial_train_rmse,final_train_rmse,initial_test_rmse,final_test_rmse
0,"Albany, NY - Chicago, IL",0.76573,0.732433,-11.2114,-5.16686,21.0959,22.5453,51.2771,36.4394
1,"Albany, NY - Orlando, FL",0.531803,0.416107,-1.39964,-0.692976,9.25274,10.3329,16.1668,13.5793
2,"Albuquerque, NM - Chicago, IL",0.504485,0.415028,-2.56934,-0.577167,8.39041,9.11637,25.179,16.7372
3,"Albuquerque, NM - Dallas, TX",1,1,1,1,2.80657e-12,3.1308e-14,3.50712e-12,7.12907e-14
4,"Albuquerque, NM - Houston, TX",0.999997,0.999996,0.999874,0.999957,0.0176341,0.0208618,0.0499627,0.0291183
...,...,...,...,...,...,...,...,...,...
399,"Seattle, WA - St. Louis, MO",0.387131,0.127598,-22.2179,-2.30261,24.7768,29.561,82.4328,31.0897
400,"Seattle, WA - Washington, DC",0.711897,0.666804,-2.47563,-1.98754,13.0253,14.0076,34.8948,32.352
401,"St. Louis, MO - Tampa, FL",1,1,1,1,8.51266e-08,5.90224e-08,0.00162509,0.00162508
402,"St. Louis, MO - Washington, DC",0.365395,0.153563,-5.89367,-3.22371,10.4563,12.076,30.6791,24.014


In [1563]:
# Engineering Additional features
only_df['initial_r2_spread'] = only_df['initial_train_r2'] - only_df['initial_test_r2']
only_df['final_r2_spread'] = only_df['final_train_r2'] - only_df['final_test_r2']
only_df['initial_rmse_spread'] = only_df['initial_train_rmse'] - only_df['initial_test_rmse']
only_df['final_rmse_spread'] = only_df['final_train_rmse'] - only_df['final_test_rmse']
only_df['test_rmse_improved'] = only_df['initial_test_rmse'] > only_df['final_test_rmse']
only_df['test_r2_improved'] = only_df['initial_test_rmse'] > only_df['final_test_rmse']
only_df['test_rmse_improvement_%'] = (only_df['initial_test_rmse'] - only_df['final_test_rmse']) / only_df['initial_test_rmse']
only_df['test_r2_improvement_%'] = (only_df['final_test_r2'] - only_df['initial_test_r2']) / only_df['initial_test_r2']
only_df['test_rmse_improvement_amount'] = only_df['initial_test_rmse'] - only_df['final_test_rmse']
only_df.head(2)

,route,initial_train_r2,final_train_r2,initial_test_r2,final_test_r2,initial_train_rmse,final_train_rmse,initial_test_rmse,final_test_rmse,initial_r2_spread,final_r2_spread,initial_rmse_spread,final_rmse_spread,test_rmse_improved,test_r2_improved,test_rmse_improvement_%,test_r2_improvement_%,test_rmse_improvement_amount
0,"Albany, NY - Chicago, IL",0.76573,0.732433,-11.2114,-5.16686,21.0959,22.5453,51.2771,36.4394,11.9772,5.89929,-30.1812,-13.8941,True,True,0.289362,-0.539144,14.8376
1,"Albany, NY - Orlando, FL",0.531803,0.416107,-1.39964,-0.692976,9.25274,10.3329,16.1668,13.5793,1.93145,1.10908,-6.91405,-3.24635,True,True,0.160052,-0.504891,2.58754


#### See below how we improved 299 models from the first model until the final model which we looped through to satify the p-value condition of 0.05
- again note earlier this does not mean we surpassed baseline

In [1564]:
# And we are please to see although the models still need work we improved our RMSE score on 299 test models 
only_df[(only_df['initial_test_rmse'] >= 0) & (only_df['initial_train_rmse'] >= 0) & (only_df['test_rmse_improved'] == True)]

,route,initial_train_r2,final_train_r2,initial_test_r2,final_test_r2,initial_train_rmse,final_train_rmse,initial_test_rmse,final_test_rmse,initial_r2_spread,final_r2_spread,initial_rmse_spread,final_rmse_spread,test_rmse_improved,test_r2_improved,test_rmse_improvement_%,test_r2_improvement_%,test_rmse_improvement_amount
0,"Albany, NY - Chicago, IL",0.76573,0.732433,-11.2114,-5.16686,21.0959,22.5453,51.2771,36.4394,11.9772,5.89929,-30.1812,-13.8941,True,True,0.289362,-0.539144,14.8376
1,"Albany, NY - Orlando, FL",0.531803,0.416107,-1.39964,-0.692976,9.25274,10.3329,16.1668,13.5793,1.93145,1.10908,-6.91405,-3.24635,True,True,0.160052,-0.504891,2.58754
2,"Albuquerque, NM - Chicago, IL",0.504485,0.415028,-2.56934,-0.577167,8.39041,9.11637,25.179,16.7372,3.07382,0.992195,-16.7886,-7.62087,True,True,0.335271,-0.775364,8.44179
3,"Albuquerque, NM - Dallas, TX",1,1,1,1,2.80657e-12,3.1308e-14,3.50712e-12,7.12907e-14,0,0,-7.00548e-13,-3.99828e-14,True,True,0.979673,0,3.43582e-12
4,"Albuquerque, NM - Houston, TX",0.999997,0.999996,0.999874,0.999957,0.0176341,0.0208618,0.0499627,0.0291183,0.000123546,3.92844e-05,-0.0323287,-0.00825647,True,True,0.4172,8.32619e-05,0.0208444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,"Seattle, WA - Spokane, WA",0.253372,0.0895097,-3.63982,-3.58877,2.6292,2.90342,7.97013,7.92617,3.89319,3.67828,-5.34093,-5.02276,True,True,0.00551563,-0.0140232,0.0439603
399,"Seattle, WA - St. Louis, MO",0.387131,0.127598,-22.2179,-2.30261,24.7768,29.561,82.4328,31.0897,22.6051,2.4302,-57.656,-1.5287,True,True,0.622848,-0.896363,51.3431
400,"Seattle, WA - Washington, DC",0.711897,0.666804,-2.47563,-1.98754,13.0253,14.0076,34.8948,32.352,3.18753,2.65435,-21.8695,-18.3444,True,True,0.0728704,-0.197156,2.5428
401,"St. Louis, MO - Tampa, FL",1,1,1,1,8.51266e-08,5.90224e-08,0.00162509,0.00162508,2.17606e-08,2.17602e-08,-0.00162501,-0.00162502,True,True,8.51856e-06,3.70814e-13,1.38435e-08


# ARIMA Model
- Taking a quick peak at our ARIMA Model
- Although this was heavily explored, ultimately I chose to focus on OLS.  The way we transformed features in OLS was a more sophisticated version of what is done in this model
- Reason being the AIC scores were similar and it was a more interesting experienced to dive into OLS statsmodel and tune each route manually via ML.
- With ARIMA not being able to calculate RMSE or R2 (make it a black box model) was something that shied us aways from it.
- We took a look at our route 'Dallas, TX - Houston, TX'

In [1565]:
# dal_hou_features
from statsmodels.tsa.arima_model import ARIMA

In [1566]:
def arima_modeler(target_variable='airfare', itinerary='Dallas, TX - Houston, TX', exog_var=None, d=1, p=5, q=5, best_aic=1000, best_p=0, best_q=0):
    
    # dataframes and features
    df_train = all_route_dict[itinerary]['train_dataframe'].copy()
    df_unseen = all_route_dict[itinerary]['test_dataframe'].copy()
    df_features = list(all_route_dict[itinerary]['train_dataframe'].columns)
    
    # train / test
    train, test = train_test_split(df_train['airfare'], test_size = 0.1, shuffle = False)

    for p in range(p):                                                    # Grid Test the ARIMA Model
        for q in range(q):
            try:
                model = ARIMA(endog = train.astype(float).dropna(), # Instantiate & Fit ARIMA model
                              order = (p, d, q), # exog = df_train[exog_var][train.index], 
                              freq = 'M').fit() 
                if model.aic < best_aic:                                  # Is current model AIC better than best_aic variable?
                    best_aic = model.aic                                  # If so, then overwrite
                    best_p = p                                            # If so, then overwrite
                    best_q = q                                            # If so, then overwrite

            except:
                pass
    
    new_model = ARIMA(endog = train.astype(float).dropna(),
                              order = (best_p, d, best_q),
                              freq = 'M').fit()
    
    print()
    print(f'Complete! Minimize AIC on train data with ARIMA({best_p}, {d}, {best_q}).')
    print()    
    print(f'Best AIC: {best_aic}.')
    print()
    
    return model, new_model

In [1567]:
# First the first model results!
dal_tex, dal_tex_new = arima_modeler()
dal_tex.summary()

/Users/tonylucci/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
/Users/tonylucci/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov


Complete! Minimize AIC on train data with ARIMA(2, 1, 2).

Best AIC: 363.7379387273425.



/Users/tonylucci/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                             ARIMA Model Results                              
==============================================================================
Dep. Variable:              D.airfare   No. Observations:                  105
Model:                 ARIMA(4, 1, 0)   Log Likelihood                -180.803
Method:                       css-mle   S.D. of innovations              1.354
Date:                Mon, 12 Oct 2020   AIC                            373.606
Time:                        20:30:56   BIC                            389.529
Sample:                    04-30-1997   HQIC                           380.058
                         - 12-31-2005                                         
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.1818      0.114      1.588      0.112      -0.043       0.406
ar.L1.D.airfare    -0.0199      0.097     -0.205      0.838      -0.211       0.171
ar.L2.D.airfare    -0.0192      0.097     -0.198      0.843      -0.209       0.170
ar.L3.D.airfare    -0.0886      0.098     -0.907      0.364      -0.280       0.103
ar.L4.D.airfare    -0.0316      0.111     -0.283      0.777      -0.250       0.187
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1            1.1333           -1.5935j            1.9554           -0.1516
AR.2            1.1333           +1.5935j            1.9554            0.1516
AR.3           -2.5359           -1.3592j            2.8772           -0.4217
AR.4           -2.5359           +1.3592j            2.8772            0.4217
-----------------------------------------------------------------------------
"""

In [1568]:
# Now the best model results
dal_tex_new.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             ARIMA Model Results                              
==============================================================================
Dep. Variable:              D.airfare   No. Observations:                  105
Model:                 ARIMA(2, 1, 2)   Log Likelihood                -175.869
Method:                       css-mle   S.D. of innovations              1.245
Date:                Mon, 12 Oct 2020   AIC                            363.738
Time:                        20:30:56   BIC                            379.662
Sample:                    04-30-1997   HQIC                           370.191
                         - 12-31-2005                                         
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.1906      0.009     21.672      0.000       0.173       0.208
ar.L1.D.airfare     1.8001      0.063     28.655      0.000       1.677       1.923
ar.L2.D.airfare    -0.8262      0.059    -13.954      0.000      -0.942      -0.710
ma.L1.D.airfare    -1.9983      0.061    -32.599      0.000      -2.118      -1.878
ma.L2.D.airfare     0.9983      0.061     16.270      0.000       0.878       1.119
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1            1.0894           -0.1535j            1.1002           -0.0223
AR.2            1.0894           +0.1535j            1.1002            0.0223
MA.1            1.0000           +0.0000j            1.0000            0.0000
MA.2            1.0017           +0.0000j            1.0017            0.0000
-----------------------------------------------------------------------------
"""

In [1569]:
# now our OLS results for the same route
all_route_dict['Dallas, TX - Houston, TX']['final_model'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                airfare   R-squared:                       0.336
Model:                            OLS   Adj. R-squared:                  0.282
Method:                 Least Squares   F-statistic:                     6.314
Date:                Mon, 12 Oct 2020   Prob (F-statistic):           2.08e-05
Time:                        20:30:56   Log-Likelihood:                -237.49
No. Observations:                  82   AIC:                             489.0
Df Residuals:                      75   BIC:                             505.8
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                          -0.0030      0.001     -4.711      0.000      -0.004      -0.002
sf_cost_per_mile_diff_1       911.6358    347.708      2.622      0.011     218.966    1604.306
sf_flight_revenue_diff_2     4.358e-06   2.08e-06      2.099      0.039    2.22e-07    8.49e-06
sf_airfare_diff_1              -4.0965      1.787     -2.293      0.025      -7.656      -0.537
sf_time_diff_1                 -1.2160      0.258     -4.711      0.000      -1.730      -0.702
sf_passengers_diff_2           -0.0004      0.000     -2.159      0.034      -0.001   -2.92e-05
sf_total_flight_cost_diff_2  2.335e-05   1.04e-05      2.255      0.027    2.73e-06     4.4e-05
dist_miles                      2.5084      0.454      5.523      0.000       1.604       3.413
==============================================================================
Omnibus:                        2.809   Durbin-Watson:                   0.181
Prob(Omnibus):                  0.246   Jarque-Bera (JB):                2.408
Skew:                           0.419   Prob(JB):                        0.300
Kurtosis:                       3.050   Cond. No.                     9.55e+21
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.9e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

### CONCLUSION / BUSINESS RECOMMENDATIONS
## WINS
- We succeeded in building 63 separate route specific that predicted R2 scores between 90-100% out of 375 total routes examined.

- From this we can be confident to tell an individual that when they are searching for a flight, they should cross reference their findings with our prediction model.  Of course it will be up to them, but they will be empowered to make a decision whether to buy this flight now, or wait until a future date dependent on their own personal budget.

- We succeed in improving RMSE scores 21.85% on 299 of our 375 total routes.  From starting with an initial featureset and using machine learning to make feature selection based on specific conditions.

- 265 out of 375 of our test R2 scores outperformed the baseline scores

## FUTURE THOUGHTS

- As we knew predicting airling pricing is a very tricky task.  Ensuring we get as much granular information as possible can be a next step to further building out this project and finding patterns to build more effective models.

- Given more time I would look to scrape the web for pricing information for flights perhaps down to the daily / Hourly level.  So much goes into airpline pricing and as we all know the time of the purchase really makes a difference.

- We'd love to build a web app where individuals can plug their price quote into it that will let them know how much savings they will make by booking the flight now, and, perhaps, show them where we seee the price going over the next 3 months so they create urgency in themselves for making the purchase.